In [1]:
!pip install wordcloud

In [3]:
from wordcloud import WordCloud

In [7]:
import glob
import joblib
import streamlit as st

@st.cache_resource
def load_models():
    vectorizer = joblib.load("models/tfidf_vectorizer.pkl")

    model_files = glob.glob("models/*.pkl")
    model_files = [m for m in model_files if "vectorizer" not in m]

    if not model_files:
        st.error("⚠️ 沒有找到模型檔案，請先執行 train_pipeline.py。")
        st.stop()

    model_path = model_files[0]
    st.info(f"✅ 已載入模型：{model_path}")
    model = joblib.load(model_path)

    return vectorizer, model


In [14]:
# 03_streamlit_app.py (進階版)

import streamlit as st
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns

# ===========================
# 1. 頁面設定
# ===========================
st.set_page_config(page_title="Spam Email Classifier", page_icon="📧", layout="wide")

st.title("📧 Spam Email Classifier")
st.caption("一個使用機器學習建立的垃圾郵件分類器 | 2025 ML Project by Beck Lin")

# ===========================
# 2. 載入模型與向量器
# ===========================
@st.cache_resource
def load_models():
    vectorizer = joblib.load("models/tfidf_vectorizer.pkl")
    model = joblib.load("models/Logistic_Regression.pkl")  # 根據你最佳模型調整名稱
    return vectorizer, model

vectorizer, model = load_models()

# ===========================
# 3. 使用者輸入區
# ===========================
st.subheader("📝 測試郵件內容")
user_input = st.text_area(
    "請輸入郵件內容：",
    height=150,
    placeholder="例如：Congratulations! You've won a $1000 gift card. Click here to claim...",
)

col1, col2 = st.columns([1, 2])

if col1.button("🔍 開始分析"):
    if user_input.strip() == "":
        st.warning("請先輸入郵件內容。")
    else:
        X_input = vectorizer.transform([user_input])
        pred = model.predict(X_input)[0]
        try:
            proba = model.predict_proba(X_input)[0][1]
        except:
            proba = None

        if pred == 1:
            col2.error("🚨 預測結果：**Spam (垃圾郵件)**")
        else:
            col2.success("✅ 預測結果：**Not Spam (正常郵件)**")

        if proba is not None:
            col2.metric("Spam 機率", f"{proba*100:.2f}%")
            st.progress(float(proba))

# ===========================
# 4. 模型效能摘要
# ===========================
st.markdown("---")
st.subheader("📊 模型效能摘要")

try:
    # 嘗試從 pipeline 產生的結果表載入 (若有)
    df_metrics = pd.read_csv("data/model_results.csv")
    st.dataframe(df_metrics.style.highlight_max(subset=["F1"], color="lightgreen"))
except:
    # 若沒有結果檔，顯示手動摘要
    st.info("""
    **模型摘要 (手動輸入)**
    - Logistic Regression：Accuracy 98%、F1 0.97
    - Naive Bayes：Accuracy 97%、F1 0.94
    - SVM：Accuracy 97%、F1 0.96
    """)

# ===========================
# 5. 詞雲可視化
# ===========================
st.markdown("---")
st.subheader("☁️ 詞雲視覺化")

try:
    data = joblib.load("data/processed_spam_data.pkl")
    X_train, X_test, y_train, y_test = data

    # 因為這裡沒有原始文字資料，我們用示意資料
    spam_texts = [
        "free winner cash prize money offer congratulations click claim now",
        "you have won lottery gift card free coupon claim reward now",
    ]
    ham_texts = [
        "see you at lunch tomorrow meeting scheduled at 3pm",
        "please find attached the report for this week project update",
    ]

    spam_wc = WordCloud(width=500, height=300, background_color="white").generate(" ".join(spam_texts))
    ham_wc = WordCloud(width=500, height=300, background_color="white").generate(" ".join(ham_texts))

    col1, col2 = st.columns(2)
    with col1:
        st.image(spam_wc.to_array(), caption="🚨 Spam 常見詞")
    with col2:
        st.image(ham_wc.to_array(), caption="✅ Ham 常見詞")

except Exception as e:
    st.warning("無法產生詞雲（可能缺少文字資料）")

# ===========================
# 6. 簡單統計圖表
# ===========================
st.markdown("---")
st.subheader("📈 Spam vs Ham 統計分析")

spam_count = 747
ham_count = 4827
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

# 圓餅圖
ax[0].pie([ham_count, spam_count], labels=["Ham", "Spam"], autopct="%1.1f%%", colors=["#4CAF50", "#F44336"])
ax[0].set_title("資料集比例")

# 長度分佈（模擬）
np.random.seed(42)
ham_lengths = np.random.normal(80, 20, 200)
spam_lengths = np.random.normal(120, 25, 200)
sns.kdeplot(ham_lengths, ax=ax[1], label="Ham")
sns.kdeplot(spam_lengths, ax=ax[1], label="Spam", color="red")
ax[1].set_title("郵件長度分佈")
ax[1].set_xlabel("字數")
ax[1].legend()

st.pyplot(fig)

# ===========================
# 7. 關於模型說明
# ===========================
st.markdown("---")
with st.expander("📘 關於此模型"):
    st.write("""
    - 模型使用 **TF-IDF 向量化** + **Logistic Regression**
    - 訓練資料來自 SMS Spam Collection Dataset
    - 評估指標：Accuracy、Precision、Recall、F1-score
    - 於 Streamlit Cloud 部署，可即時預測郵件內容是否為垃圾郵件。
    """)

st.markdown("—")
st.caption("🧠 Created by Beck Lin | 2025 Machine Learning Project")


FileNotFoundError: [Errno 2] No such file or directory: 'models/Logistic_Regression.pkl'

In [20]:
streamlit run 03_streamlit_app.py

SyntaxError: invalid decimal literal (120509875.py, line 1)